# Creating An AI-Based JFK Speech Writer: Part 1
---------------

## Introduction
----------------
One of the most quitensential projects to complete when getting started with Deep Learning and Natural Language Processing is with text generation with Recurrent Neural Networks. The internet is littered with examples of people training on books of Shakespeare and using the network to generate new text that mimics Shakespeare's style. I wanted to do something along these lines, but a little more creative. Many would agree one of the best orators of all time would have to be John F. Kennedy. I am a personally a big nerd of an President Kennedy's speaches and spent many hours listening to his words. So I started this project to see if could write a neural network to generate a Kennedy speech writer. 

Speeches writen by the President, Senators & Representatives (JFK was all 3) are under the public domain which means they are publicly available! In this first part, I will quickly go over how I was able to create a training set of JFK's speeches. 

## Creating A Database Of Speeches With BeautifulSoup
-------------------

The first place I went to get President Kennedy's speeches was his [Presidential Library's Website](https://jfklibrary.org/). The website has a list of each 


![scrape](images/scrape.png)

In [55]:
import requests
import os
from bs4 import BeautifulSoup


In [56]:
def get_links():
    """
    Used 
    """
    address = "https://www.jfklibrary.org/archives/other-resources/john-f-kennedy-speeches"
    page_nbs = range(13,38)

    links = []
    for num in page_nbs:
        page = requests.get(f"{address}?page={num}")
        soup = BeautifulSoup(page.text, 'html.parser')
        links.extend([article.find("a")["href"] for article in soup.find_all("article")])
    
    return links


In [57]:
links = get_links()

In [58]:
link = links[0]
print(link)

/archives/other-resources/john-f-kennedy-speeches/indianapolis-in-19591002


In [59]:
title = link.partition("/john-f-kennedy-speeches/")[2]
print(title)

indianapolis-in-19591002


In [ ]:
# pip install google-cloud-storage
from google.cloud import storage
client = storage.Client()
bucket = client.create_bucket('harmon-kennedy')


In [61]:
def speech_etl(link: str) -> None:
    site = f"https://www.jfklibrary.org/{link}"
    page = requests.get(site)
    soup = BeautifulSoup(page.text, 'html.parser')
    title = link.partition("/john-f-kennedy-speeches/")[2]
    file_name = f"{title}.txt"
    blob = bucket.blob(file_name)
    speech = soup.find("article").text
    blob.upload_from_string(data=speech, content_type='text/plain')
    
    

In [62]:
for link in links:
    speech_etl(link)

## Next Steps
-----------------------